In [ ]:
import time

focus_scores = []
start_time = time.time()


In [4]:
import numpy as np
def calculate_focus_score(landmarks, image_w, image_h):
    def to_px(pt): return np.array([int(pt.x * image_w), int(pt.y * image_h)])

    l_eye = to_px(landmarks[33])
    r_eye = to_px(landmarks[263])
    nose = to_px(landmarks[1])
    l_ear = to_px(landmarks[234])
    r_ear = to_px(landmarks[454])
    top_lid = to_px(landmarks[159])
    bottom_lid = to_px(landmarks[145])
    mouth_upper = to_px(landmarks[13])
    mouth_lower = to_px(landmarks[14])

    eye_dist = np.linalg.norm(l_eye - r_eye)
    head_tilt = abs(l_ear[1] - r_ear[1])
    nose_offset = abs((l_eye[0] + r_eye[0]) / 2 - nose[0])

    # Eye aspect ratio (simplified)
    eye_open = np.linalg.norm(top_lid - bottom_lid)
    mouth_open = np.linalg.norm(mouth_upper - mouth_lower)

    focus = 100
    if nose_offset > 40:
        focus -= 20  # not centered
    if head_tilt > 30:
        focus -= 20  # head tilted
    if eye_dist < 60:
        focus -= 10  # maybe looking away
    if eye_open < 5:
        focus -= 20  # closed eyes
    if mouth_open > 15:
        focus -= 20  # talking (approx)

    return max(0, min(100, focus))


In [5]:
duration_minutes = 30  # or 60 for an hour

while True:
    ret, frame = cap.read()
    if not ret:
        break
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
            focus = calculate_focus_score(face_landmarks.landmark, w, h)

            focus_scores.append(focus)

            # Show real-time
            label = f"Focus: {int(focus)}%"
            color = (0, 255, 0) if focus >= 70 else (0, 0, 255)
            cv2.putText(frame, label, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    elapsed_min = (time.time() - start_time) / 60
    cv2.putText(frame, f"Time: {int(elapsed_min)} min", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    # Show average every 5 minutes or at end
    if elapsed_min >= duration_minutes:
        avg_focus = sum(focus_scores) / len(focus_scores)
        print(f"\n🧠 Session Complete: Average Focus = {avg_focus:.2f}%\n")
        break

    cv2.imshow("Student Focus Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


NameError: name 'cap' is not defined

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'timestamp': np.linspace(0, len(focus_scores)*0.05, len(focus_scores)),  # assuming ~20 FPS
    'focus': focus_scores
})
df.to_csv("focus_session_log.csv", index=False)
